In [1]:
!pip install -U segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 8.9 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=62000c286f6eed0a41cea6042886327184fd308cf37ce32b8d6617e335d09774
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=15a68d50cf6adbee6ff079af89881fa44271e5d9739f0cc67180369b5eb143a0
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [2]:
!pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.6/290.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp
import os
import numpy as np
from glob import glob
import json
from torchvision import transforms
from PIL import Image
import torch
import argparse
import warnings
import json
import os

In [4]:

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import cv2 as cv
from sklearn.model_selection import train_test_split
from torchvision.transforms import ToTensor
import random
from glob import glob

class RetinaDataset(Dataset):
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.to_tensor = ToTensor()
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):

        image = cv.imread(self.images[idx], cv.IMREAD_COLOR)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        mean = image.mean(axis=(0, 1))
        std = image.std(axis=(0, 1))
        image = (image - mean[None, None, :]) / std[None, None, :]
        image = image.astype(np.float32)

        mask = cv.imread(self.masks[idx], cv.IMREAD_GRAYSCALE)
        mask[mask > 0] = 1
        mask = mask.astype(np.float32)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image, mask = augmented["image"], augmented["mask"]

        image_tensor = self.to_tensor(image)
        mask_tensor = self.to_tensor(mask)

        return image_tensor, mask_tensor


def load_data(base_directory="/kaggle/input/retina-blood-vessel/Data", batch_size=4, train_transform=None, test_transform=None):
    train_images = sorted(glob(os.path.join(base_directory, "train", "image", "*.png")))
    train_masks = sorted(glob(os.path.join(base_directory, "train", "mask", "*.png")))
    test_images = sorted(glob(os.path.join(base_directory, "test", "image", "*.png")))
    test_masks = sorted(glob(os.path.join(base_directory, "test", "mask", "*.png")))

    train_images, val_images, train_masks, val_masks = train_test_split(
        train_images, train_masks, test_size=0.25, random_state=5
    )

    train_dataset = RetinaDataset(train_images, train_masks, transform=train_transform)
    val_dataset = RetinaDataset(val_images, val_masks, transform=test_transform)
    test_dataset = RetinaDataset(test_images, test_masks, transform=test_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    return train_loader, val_loader, test_loader

In [5]:
def set_seed():
    torch.manual_seed(5)
    torch.cuda.manual_seed(5)
    torch.cuda.manual_seed_all(5)
    np.random.seed(5)
    random.seed(5)

In [6]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import os
import numpy as np
from glob import glob
import albumentations as A


def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
    )

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.criterion = nn.BCEWithLogitsLoss()

        self.enc1 = conv_block(in_channels, 64)
        self.enc2 = conv_block(64, 128)
        self.enc3 = conv_block(128, 256)
        self.enc4 = conv_block(256, 512)
        self.enc5 = conv_block(512, 1024)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)

        self.dec1 = conv_block(1024, 512)
        self.dec2 = conv_block(512, 256)
        self.dec3 = conv_block(256, 128)
        self.dec4 = conv_block(128, 64)

        self.out = nn.Conv2d(64, out_channels, kernel_size=1)

        self.train_loss_sum = 0.0
        self.train_iou_sum = 0.0
        self.train_batches = 0
        self.val_loss_sum = 0.0
        self.val_iou_sum = 0.0
        self.val_batches = 0
        self.train_dice_sum = 0.0
        self.train_acc_sum = 0.0
        self.val_dice_sum = 0.0
        self.val_acc_sum = 0.0

    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.enc2(self.pool(x1))
        x3 = self.enc3(self.pool(x2))
        x4 = self.enc4(self.pool(x3))
        x5 = self.enc5(self.pool(x4))

        x = self.upconv4(x5)
        x = torch.cat([x, x4], dim=1)
        x = self.dec1(x)

        x = self.upconv3(x)
        x = torch.cat([x, x3], dim=1)
        x = self.dec2(x)

        x = self.upconv2(x)
        x = torch.cat([x, x2], dim=1)
        x = self.dec3(x)

        x = self.upconv1(x)
        x = torch.cat([x, x1], dim=1)
        x = self.dec4(x)

        x = self.out(x)
        return x
    def common_step(self, batch, batch_idx, device="cuda"):
        image, mask = batch
        image = image.to(device)
        mask = mask.to(device)
        pred = self.forward(image)
        loss = self.criterion(pred, mask)
        pred = torch.sigmoid(pred)
        mask = mask.round().long()
        tp, fp, fn, tn = smp.metrics.get_stats(pred, mask, mode="binary", threshold=0.5)  # type: ignore
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro").item()
        dice = (2 * tp.sum() / (2 * tp.sum() + fp.sum() + fn.sum())).item()
        acc = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro").item()
        return loss, iou_score, dice, acc

    def training_step(self, batch, batch_idx, device="cuda"):
        loss, iou_score, dice, acc = self.common_step(batch, batch_idx, device="cuda")
        self.train_loss_sum += loss
        self.train_iou_sum += iou_score
        self.train_dice_sum += dice
        self.train_acc_sum += acc
        self.train_batches += 1
        return loss,iou_score, dice, acc

    def on_train_epoch_end(self):
        avg_train_loss = self.train_loss_sum / self.train_batches
        avg_train_iou = self.train_iou_sum / self.train_batches
        avg_train_dice = self.train_dice_sum / self.train_batches
        avg_train_acc = self.train_acc_sum / self.train_batches
        self.train_loss_sum = 0.0
        self.train_iou_sum = 0.0
        self.train_dice_sum = 0.0
        self.train_acc_sum = 0.0
        self.train_batches = 0
        return avg_train_loss, avg_train_iou, avg_train_dice, avg_train_acc


    def validation_step(self, batch, batch_idx):
        loss, iou_score, dice, acc = self.common_step(batch, batch_idx, device="cuda")
        self.val_loss_sum += loss
        self.val_iou_sum += iou_score
        self.val_dice_sum += dice
        self.val_acc_sum += acc
        self.val_batches += 1
        return loss,iou_score, dice, acc

    def on_validation_epoch_end(self):
        avg_val_loss = self.val_loss_sum / self.val_batches
        avg_val_iou = self.val_iou_sum / self.val_batches
        avg_val_dice = self.val_dice_sum / self.val_batches
        avg_val_acc = self.val_acc_sum / self.val_batches
        self.val_loss_sum = 0.0
        self.val_iou_sum = 0.0
        self.val_dice_sum = 0.0
        self.val_acc_sum = 0.0
        self.val_batches = 0
        return avg_val_loss, avg_val_iou, avg_val_dice, avg_val_acc

def train_model(model, train_loader, val_loader, optimizer, num_epochs=800, patience=50, device="cuda"):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    best_val_iou = float("-inf")
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_iou = 0.0, 0.0
        # enable gradient calculation
        torch.set_grad_enabled(True)

        for batch_idx, batch in enumerate(train_loader):
            train_loss, train_iou, train_dice, train_acc = model.training_step(batch, batch_idx)

            loss = train_loss  #loss scaled down by accumulation steps
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            if batch_idx % 5 == 0:
                torch.set_grad_enabled(False)
                model.eval()
                # Validation loop
                for val_batch_idx, val_batch in enumerate(val_loader):
                    val_loss, val_iou, val_dice, val_acc = model.validation_step(val_batch, val_batch_idx)
                torch.set_grad_enabled(True)
                model.train()

        train_loss, train_iou, train_dice, train_acc = model.on_train_epoch_end()
        val_loss,val_iou, val_dice, val_acc = model.on_validation_epoch_end()
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train IoU: {train_iou:.4f}, Train Dice: {train_dice:.4f} , Train Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation IoU: {val_iou:.4f}, Validation Dice: {val_dice:.4f} , Validation Accuracy: {val_acc:.4f}")
        # print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train IoU: {train_iou:.4f}")
        torch.save(model.state_dict(), "unet_seg.pth")

        if val_iou > best_val_iou:
                best_val_iou = val_iou
                patience_counter = 0
                torch.save(model.state_dict(), "best_iou_seg.pth")
        else:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping triggered")
                    break

if __name__ == "__main__":
    set_seed()
    train_transform = A.Compose(
      [
          A.Resize(512, 512),
          A.HorizontalFlip(p=0.5),
          A.VerticalFlip(p=0.5),
          A.Rotate(limit=30, p=0.5),
          A.Sharpen(alpha=(0.5, 0.9), lightness=(0.5, 1.0), p=0.3),
          A.Emboss(alpha=(0.1, 0.3), strength=(0.5, 1.0), p=0.3),
      ]
    )
    test_transform = A.Compose(
      [
          A.Resize(512, 512),
      ]
    )

    train_loader, val_loader, test_loader = load_data(base_directory="/kaggle/input/retina-blood-vessel/Data",train_transform=train_transform)
    model = UNet(in_channels=3, out_channels=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    train_model(model, train_loader, val_loader, optimizer)


Epoch 1, Train Loss: 0.7086, Train IoU: 0.1243, Train Dice: 0.2204 , Train Accuracy: 0.2340, Validation Loss: 0.7063, Validation IoU: 0.1442, Validation Dice: 0.2511 , Validation Accuracy: 0.2427
Epoch 2, Train Loss: 0.5431, Train IoU: 0.0001, Train Dice: 0.0003 , Train Accuracy: 0.8758, Validation Loss: 0.4761, Validation IoU: 0.0001, Validation Dice: 0.0002 , Validation Accuracy: 0.8749
Epoch 3, Train Loss: 0.4110, Train IoU: 0.0000, Train Dice: 0.0000 , Train Accuracy: 0.8781, Validation Loss: 0.3620, Validation IoU: 0.0000, Validation Dice: 0.0000 , Validation Accuracy: 0.8750
Epoch 4, Train Loss: 0.3751, Train IoU: 0.0000, Train Dice: 0.0000 , Train Accuracy: 0.8787, Validation Loss: 0.3492, Validation IoU: 0.0000, Validation Dice: 0.0000 , Validation Accuracy: 0.8750
Epoch 5, Train Loss: 0.3588, Train IoU: 0.0000, Train Dice: 0.0000 , Train Accuracy: 0.8780, Validation Loss: 0.3449, Validation IoU: 0.0000, Validation Dice: 0.0000 , Validation Accuracy: 0.8750
Epoch 6, Train Loss: